#PROBLEM STATEMENT
The language Hinglish involves a hybrid mixing of Hindi and English within conversations, individual sentences and even words. An example: “nahi mei nahi aa sakta”. Translation: “no, I cannot come.” It is gaining popularity as a way of speaking that demonstrates you are modern, yet locally grounded. India being a diverse country, one witnesses both ends of the spectrum. Namely, a hindi speaking commoner who can read and understand the devanagari script. And tourists coming in from abroad, who may or may not understand the language entirely. Since the local indian markets which are a huge source of attraction for foreign tourists, comprise a majority of purely devanagari understanding vendors, we found the need to use Hinglish as a medium for the two extreme ends to meet.

Our program aims to detect two things:
Whether the word typed is either english or hindi written in english
To translate the hinglish word to both: English and Hindi

### English to Hindi Transliteration Tutorial

#### GPU

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


#### Download Data

In [2]:
!mkdir data
!wget -N "https://raw.githubusercontent.com/bsantraigi/tensorflow-seq2seq-hindi/master/data/Hindi%20-%20Word%20Transliteration%20Pairs%201.txt" -P data/

--2021-04-12 22:17:35--  https://raw.githubusercontent.com/bsantraigi/tensorflow-seq2seq-hindi/master/data/Hindi%20-%20Word%20Transliteration%20Pairs%201.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 773211 (755K) [text/plain]
Saving to: ‘data/Hindi - Word Transliteration Pairs 1.txt’

Hindi - Word Transl 100%[===================>] 755.09K  --.-KB/s    in 0.02s   

Last-modified header missing -- time-stamps turned off.
2021-04-12 22:17:35 (43.9 MB/s) - ‘data/Hindi - Word Transliteration Pairs 1.txt’ saved [773211/773211]



#### Import Stuff

In [3]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7MB 43kB/s 
     |████████████████████████████████| 3.2MB 33.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 368kB 43.5MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [4]:
!pip install indic-transliteration


     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 911kB 33.9MB/s 
     |████████████████████████████████| 665kB 47.9MB/s 
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20


In [5]:
import nltk
from collections import Counter
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
from tensorflow.contrib import seq2seq
from tensorflow.contrib.rnn import DropoutWrapper
import random

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Global Parameters

In [7]:
MAX_SEQ_LEN = 20
BATCH_SIZE = 64

### Language Vocabulary 
* (Vocab of characters, i.e. an Alphabet)

In [8]:
class Lang:
    def __init__(self, counter, vocab_size):
        self.word2id = {}
        self.id2word = {}
        self.pad = "<PAD>"
        self.sos = "<SOS>"
        self.eos = "<EOS>"
        self.unk = "<UNK>"
        
        self.ipad = 0
        self.isos = 1
        self.ieos = 2
        self.iunk = 3
        
        self.word2id[self.pad] = 0
        self.word2id[self.sos] = 1
        self.word2id[self.eos] = 2
        self.word2id[self.unk] = 3
        
        self.id2word[0] = self.pad
        self.id2word[1] = self.sos
        self.id2word[2] = self.eos
        self.id2word[3] = self.unk
        
        curr_id = 4
        for w, c in counter.most_common(vocab_size):
            self.word2id[w] = curr_id
            self.id2word[curr_id] = w
            curr_id += 1
            
    def encodeSentence(self, s, max_len=-1):
        wseq = s.lower().strip()
        if max_len == -1:
            return [self.word2id[w] if w in self.word2id else self.iunk for w in wseq]
        else:
            return ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + [self.ieos] + [self.ipad]*max_len)[:max_len]
        
    def encodeSentence2(self, s, max_len=-1):
        wseq = wseq = s.lower().strip()
        return min(max_len, len(wseq)+1), \
            ([self.word2id[w] if w in self.word2id else self.iunk for w in wseq] + \
                [self.ieos] + [self.ipad]*max_len)[:max_len]
    
    def decodeSentence(self, id_seq):
        id_seq = np.array(id_seq + [self.ieos])
        j = np.argmax(id_seq==self.ieos)
        s = ''.join([self.id2word[x] for x in id_seq[:j]])
        s = s.replace(self.unk, "UNK")
        return s

In [9]:
# Total number of samples to read
N = 30823

### Reading the data files
- Each line contains a hindi word in both English and Devnagari script

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
hi_counter = Counter()
hi_sentences=[]
en_counter = Counter()
en_sentences=[]
#with open("Hindi - Word Transliteration Pairs 1.txt") as f:

with open("/content/drive/MyDrive/Hinglish/Hindi - Word Transliteration Pairs 1.txt") as f:
    for line in tqdm_notebook(f, total=N, desc="Reading file:"):
        en, hi = line.strip().split("\t")
        hi_sentences.append(hi)
        en_sentences.append(en)
    for line in tqdm_notebook(hi_sentences, desc="Processing inputs:"):
        for w in line.strip():
            hi_counter[w] += 1
    for line in tqdm_notebook(en_sentences, desc="Processing inputs:"):
        for w in line.strip():
            en_counter[w] += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [12]:
# A few sample hindi characters
print("Most common hi characters in dataset:\n", hi_counter.most_common(5))

print("\nTotal (hi)characters gathered from dataset:",len(hi_counter))

# A few sample english characters
print("\nMost common en characters in dataset:\n", en_counter.most_common(5))

print("\nTotal (en)characters gathered from dataset:", len(en_counter))

Most common hi characters in dataset:
 [('ा', 21123), ('र', 9205), ('े', 8100), ('न', 7225), ('ी', 6546)]

Total (hi)characters gathered from dataset: 66

Most common en characters in dataset:
 [('a', 57220), ('n', 15015), ('i', 14015), ('h', 13805), ('e', 12264)]

Total (en)characters gathered from dataset: 27


In [13]:
en_lang = Lang(en_counter, len(en_counter))
hi_lang = Lang(hi_counter, len(hi_counter))

In [14]:
print(en_lang)
print(hi_lang)

In [15]:
print("Test en encoding:", en_lang.encodeSentence("Shukriya"))

print("Test en decoding:", en_lang.decodeSentence(en_lang.encodeSentence("Shukriya", 10)))

print("Test hindi encoding:", hi_lang.encodeSentence("शुक्रिया", 10))

print("Test hindi decoding:", hi_lang.decodeSentence((hi_lang.encodeSentence("शुक्रिया", 10))))

Test en encoding: [15, 7, 10, 13, 9, 6, 20, 4]
Test en decoding: shukriya
Test hindi encoding: [35, 19, 15, 22, 5, 12, 21, 4, 2, 0]
Test hindi decoding: शुक्रिया


In [16]:
VE = len(en_lang.word2id)
VH = len(hi_lang.word2id)

# The Seq2Seq architecture
Seq2Seq is a method of encoder-decoder based machine translation and language processing that maps an input of sequence to an output of sequence with a tag and attention value. The idea is to use 2 RNNs that will work together with a special token and try to predict the next state sequence from the previous sequence.

A typical sequence to sequence model has two parts – an encoder and a decoder. Both the parts are practically two different neural network models combined into one giant network. 
Broadly, the task of an encoder network is to understand the input sequence, and create a smaller dimensional representation of it. This representation is then forwarded to a decoder network which generates a sequence of its own that represents the output. Let’s take an example of a conversational agent to understand the concept.

- We will implement a seq2seq architecture for transliteration in Tensorflow r1.13.1 / r1.14
- Debugging Tip: Always keep track of tensor dimensions!
- **Tensorflow Computation Graph** - We will build a tf computation graph first. This is the representation used by tf for any neural network architecture. Once the computation graph is built, you can feed data to it for training or inference

#### Character Embedding Matrix

In [17]:
en_word_emb_matrix = tf.get_variable("en_word_emb_matrix", (VE, 300), dtype=tf.float32)
hi_word_emb_matrix = tf.get_variable("hi_word_emb_matrix", (VH, 300), dtype=tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


#### Placeholders
- Input to a tensorflow graph is 

In [18]:
keep_prob = tf.placeholder(tf.float32)

input_ids = tf.placeholder(tf.int32, (None, MAX_SEQ_LEN))
input_lens = tf.placeholder(tf.int32, (None, ))

ph_target_ids = tf.placeholder(tf.int32, (None, MAX_SEQ_LEN))
target_lens = tf.placeholder(tf.int32, (None, ))

In [19]:
# Add SOS or GO symbol
target_ids = tf.concat([tf.fill([BATCH_SIZE,1], hi_lang.isos), ph_target_ids], -1)

#### Building the computation graph

In [20]:
input_emb = tf.nn.embedding_lookup(en_word_emb_matrix, input_ids)
target_emb = tf.nn.embedding_lookup(hi_word_emb_matrix, target_ids[:, :-1])

In [21]:
input_emb.shape

TensorShape([Dimension(None), Dimension(20), Dimension(300)])

#### Encoder - RNN based sequence encoder

In [22]:
encoder_cell = tf.nn.rnn_cell.GRUCell(128) # 128 is the dimension of hidden state
encoder_cell = DropoutWrapper(encoder_cell, output_keep_prob=keep_prob) # Adding Dropout for regularization

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


In [23]:
enc_outputs, enc_state = tf.nn.dynamic_rnn(
    encoder_cell, # The encoder GRU cell
    input_emb, # Embedded input sequence
    sequence_length=input_lens, # Sequence lengths of individual inputs in a batch
    initial_state=encoder_cell.zero_state(BATCH_SIZE, dtype=tf.float32)
)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [24]:
# Confirm the shape of the final hidden state
enc_state.shape

TensorShape([Dimension(64), Dimension(128)])

#### Decoder

In [25]:
decoder_cell = tf.nn.rnn_cell.GRUCell(128)
decoder_cell = DropoutWrapper(decoder_cell, output_keep_prob=keep_prob)

#### Decoder to Output Vocab Projection Layer

In [26]:
output_projection = tf.layers.Dense(len(hi_lang.word2id))

#### Decoder Training Helper

In [27]:
helper = seq2seq.TrainingHelper(target_emb, target_lens)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, enc_state, output_projection)
outputs, _, outputs_lens = seq2seq.dynamic_decode(decoder, maximum_iterations=MAX_SEQ_LEN, 
                                                  impute_finished=False, swap_memory=True)
output_max_len = tf.reduce_max(outputs_lens)

#### And Decoder Inference Helper

In [28]:
# Using the decoder_cell without dropout here.
infer_helper = seq2seq.GreedyEmbeddingHelper(hi_word_emb_matrix, tf.fill([BATCH_SIZE, ], hi_lang.isos), hi_lang.ieos)
infer_decoder = seq2seq.BasicDecoder(decoder_cell, infer_helper, enc_state, output_projection)
infer_output = seq2seq.dynamic_decode(infer_decoder, maximum_iterations=MAX_SEQ_LEN, swap_memory=True)

#### Loss and Optimizers

In [29]:
# Sequence mask:
# To make sure we don't back-propagate error from output of length positions
masks = tf.sequence_mask(target_lens, output_max_len, dtype=tf.float32, name='masks')

# Loss function - weighted softmax cross entropy
cost = seq2seq.sequence_loss(
    outputs[0],
    target_ids[:, 1:(output_max_len + 1)],
    masks)

# Optimizer
optimizer = tf.train.AdamOptimizer(0.0001)

In [30]:
train_op = optimizer.minimize(cost)

In [31]:
init = tf.global_variables_initializer()

#### Tensorflow Sessions

In [32]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True

In [33]:
sess = tf.InteractiveSession(config=sess_config)
sess.run(init)

#### Minibatch Training + Validation
- Performance Evaluation using BLEU scores

In [34]:
random.seed(41)

In [35]:
parallel = list(zip(en_sentences, hi_sentences))

In [36]:
random.shuffle(parallel)

In [37]:
parallel[123]

('kijiyega', 'कीजियेगा')

In [38]:
train_n = int(0.95*N)
valid_n = N - train_n

In [39]:
train_pairs = parallel[:train_n].copy()
valid_pairs = parallel[train_n:]

In [40]:
def small_test():
    all_bleu = []
    smoothing = nltk.translate.bleu_score.SmoothingFunction().method7
    for m in range(0, valid_n, BATCH_SIZE):
        # print(f"Status: {m}/{N}", end='\r')
        n = m + BATCH_SIZE
        if n > valid_n:
            # print("Epoch Complete...")
            break

        input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
        input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
        for i in range(m, n):
            b,a = en_lang.encodeSentence2(valid_pairs[i][0], MAX_SEQ_LEN)
            input_batch[i-m,:] = a
            input_lens_batch[i-m] = b

    #     target_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
    #     target_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
    #     for i in range(m, n):
    #         b,a = hi_lang.encodeSentence2(valid_pairs[i][1], MAX_SEQ_LEN)
    #         target_batch[i-m,:] = a
    #         target_lens_batch[i-m] = b

        feed_dict={
            input_ids: input_batch,
            input_lens: input_lens_batch,
            #target_ids: target_batch,
            #target_lens: target_lens_batch,
            keep_prob: 1.0
        }
        pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
        for k, pred_ in enumerate(pred_batch):
            pred_s = hi_lang.decodeSentence(list(pred_))
            ref = valid_pairs[m+k][1]
            try:
                _bx = nltk.translate.bleu_score.sentence_bleu(
                    [ref],
                    pred_s,
                    weights=[1/4]*4,
                    smoothing_function=smoothing)
            except ZeroDivisionError:
                _bx = 0
            all_bleu.append(_bx)

    print(f"BLEU Score: {np.mean(all_bleu)}")

In [41]:
for _e in range(10):
    # Mix things up a bit.
    random.shuffle(train_pairs)
    pbar = tqdm_notebook(range(0, train_n, BATCH_SIZE))
    batch_loss = 0
    bxi = 0
    for m in pbar:
        n = m + BATCH_SIZE
        if n <= train_n:
            # print("Epoch Complete... \n")

            input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            for i in range(m, n):
                b,a = en_lang.encodeSentence2(train_pairs[i][0], MAX_SEQ_LEN)
                input_batch[i-m,:] = a
                input_lens_batch[i-m] = b

            target_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
            target_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
            for i in range(m, n):
                b,a = hi_lang.encodeSentence2(train_pairs[i][1], MAX_SEQ_LEN)
                target_batch[i-m,:] = a
                target_lens_batch[i-m] = b

            feed_dict={
                input_ids: input_batch,
                input_lens: input_lens_batch,
                ph_target_ids: target_batch,
                target_lens: target_lens_batch,
                keep_prob: 0.8 
            }
            sess.run(train_op, feed_dict=feed_dict)
            batch_loss += sess.run(cost, feed_dict=feed_dict)
            pbar.set_description(f"Epoch: {_e} >> Loss: {batch_loss/(bxi+1):2.2F}:")
            bxi += 1
            if (1 + n//BATCH_SIZE) % 100 == 0:
                small_test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


BLEU Score: 0.0
BLEU Score: 0.06838558288235048
BLEU Score: 0.10599391144344654
BLEU Score: 0.134205212864294



BLEU Score: 0.18399427636824583
BLEU Score: 0.21232150718085682
BLEU Score: 0.22985928735986186
BLEU Score: 0.24334757012334626



BLEU Score: 0.26880910760111143
BLEU Score: 0.2731656796942811
BLEU Score: 0.2847091788019427
BLEU Score: 0.29270622406059055



BLEU Score: 0.30116652369085845
BLEU Score: 0.3097188066009022
BLEU Score: 0.3132556994869862
BLEU Score: 0.31475088109802446



...

BLEU Score: 0.34218557429139446
BLEU Score: 0.35365306744227637
BLEU Score: 0.35973911168531264
BLEU Score: 0.3622143874721217



BLEU Score: 0.3644332815251898
BLEU Score: 0.37224316388202555
BLEU Score: 0.3774720615204538
BLEU Score: 0.3871267228911461



BLEU Score: 0.39626761545062933
BLEU Score: 0.3976882221683209
BLEU Score: 0.4034818756427811
BLEU Score: 0.4083851704005243



BLEU Score: 0.41448196777220253
BLEU Score: 0.42418853958577135
BLEU Score: 0.4352989669542479
BLEU Score: 0.4342613488648752



BLEU Score: 0.4488719948349764
BLEU Score: 0.44732904479517693
BLEU Score: 0.45101506244910533
BLEU Score: 0.46417481166995495



In [42]:
#saver = tf.train.Saver()


In [43]:
'''sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'my_test_model')'''

"sess = tf.Session()\nsess.run(tf.global_variables_initializer())\nsaver.save(sess, 'my_test_model')"

In [44]:
'''sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('my_test_model.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
print("Model restored.")    

graph = tf.get_default_graph()


input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
b,a = en_lang.encodeSentence2('raama', MAX_SEQ_LEN)
input_batch[0, :] = a
input_lens_batch[0] = b

feed_dict={
    input_ids: input_batch,
    input_lens: input_lens_batch,
    #target_ids: target_batch,
    #target_lens: target_lens_batch,
    keep_prob: 1.0
}


tf.local_variables_initializer().run()

pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
pred_ = pred_batch[0]
pred_s = hi_lang.decodeSentence(list(pred_))
# ref = valid_pairs[m+k][1]
print(pred_s)'''

'sess=tf.Session()    \n#First let\'s load meta graph and restore weights\nsaver = tf.train.import_meta_graph(\'my_test_model.meta\')\nsaver.restore(sess,tf.train.latest_checkpoint(\'./\'))\nprint("Model restored.")    \n\ngraph = tf.get_default_graph()\n\n\ninput_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)\ninput_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)\nb,a = en_lang.encodeSentence2(\'raama\', MAX_SEQ_LEN)\ninput_batch[0, :] = a\ninput_lens_batch[0] = b\n\nfeed_dict={\n    input_ids: input_batch,\n    input_lens: input_lens_batch,\n    #target_ids: target_batch,\n    #target_lens: target_lens_batch,\n    keep_prob: 1.0\n}\n\n\ntf.local_variables_initializer().run()\n\npred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)\npred_ = pred_batch[0]\npred_s = hi_lang.decodeSentence(list(pred_))\n# ref = valid_pairs[m+k][1]\nprint(pred_s)'

In [45]:
input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
b,a = en_lang.encodeSentence2('raama', MAX_SEQ_LEN)
input_batch[0, :] = a
input_lens_batch[0] = b

feed_dict={
    input_ids: input_batch,
    input_lens: input_lens_batch,
    #target_ids: target_batch,
    #target_lens: target_lens_batch,
    keep_prob: 1.0
}
pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
pred_ = pred_batch[0]
pred_s = hi_lang.decodeSentence(list(pred_))
# ref = valid_pairs[m+k][1]
print(pred_s)

रामा


### Let's see some real translation examples now!

In [46]:
def transliterate(s):
  
      input_batch = np.zeros((BATCH_SIZE, MAX_SEQ_LEN), dtype=np.int32)
      input_lens_batch = np.zeros((BATCH_SIZE,), dtype=np.int32)
      b,a = en_lang.encodeSentence2(s, MAX_SEQ_LEN)
      input_batch[0, :] = a
      input_lens_batch[0] = b
      
      feed_dict={
          input_ids: input_batch,
          input_lens: input_lens_batch,
          #target_ids: target_batch,
          #target_lens: target_lens_batch,
          keep_prob: 1.0
      }
      pred_batch = sess.run(infer_output[0].sample_id, feed_dict=feed_dict)
      pred_ = pred_batch[0]
      pred_s = hi_lang.decodeSentence(list(pred_))
      # ref = valid_pairs[m+k][1]
      return pred_s

In [47]:
transliterate("saya")

'साया'

In [48]:
transliterate("raama")

'रामा'

In [49]:
transliterate("shahar")

'चारम'

In [50]:
transliterate("teraa")

'तरा'

In [51]:
transliterate("huuaa")

'हुाहा'

In [52]:
transliterate("pyaasa")

'पायास'

#**HINDI TO ENGLISH**

In [53]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.3MB 106kB/s 
     |████████████████████████████████| 491kB 42.1MB/s 
     |████████████████████████████████| 3.2MB 42.9MB/s 
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow 1.13.2
    Uninstalling tensorflow-1.13.2:
      Successfully uninstalled tensorflow-1.13.2


In [54]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)


ValueError: ignored

In [ ]:
file_path ="/content/drive/MyDrive/Hinglish/hin.txt"

# For testing if the file is loaded successfully, we can print few lines from the file using:
temp = open(file_path, encoding='UTF-8').read().strip().split('\n')
temp[0:5]
len(temp)

In [ ]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  
  # We need to add space between every word and punctuation such as : "Wow! It is done." -> "Wow ! It is done ."
  # This is because out network recognises these characters as different words.
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
    
  # We also need every thing with space except alphabets, punctuations and assisting symbols
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.rstrip().strip()
  
  # Adding a start and end token to the sentence
  w = '<start> ' + w + ' <end>'
  return w


In [ ]:
def create_dataset(path, num_examples):
  lines =  open(path, encoding='UTF-8').read().strip().split('\n')
  
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]
  
  return word_pairs


In [ ]:
class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))

    self.vocab = sorted(self.vocab)

    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1

    for word, index in self.word2idx.items():
      self.idx2word[index] = word

In [ ]:
pairs = create_dataset(file_path, num_examples)

In [ ]:
for hi in pairs:
  print(hi)

In [ ]:
def max_length(tensor):
  return max(len(t) for t in tensor)

def load_dataset(path, num_examples):
  # creating cleaned input and output pairs
  pairs = create_dataset(path, num_examples)
  
  # index language using the class defined above
  for en,hi in pairs:
    inp_lang = LanguageIndex(hi)
    targ_lang = LanguageIndex(en)
  
  # Vectorize the input and target languages
  
  # Hindi sentences
  input_tensor = [[inp_lang.word2idx[s] for s in hi.split(' ')] for en, hi in pairs]
  
  # English sentences
  target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
  
  # We set input vector size to the size of longest sentence (sentence with maximum length) in out dataset.
  max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
  # We need to add padding other sentences before using them. We will pad the input and output tensor to max-length
  input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, maxlen=max_length_inp, padding='post')
  
  target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, maxlen=max_length_tar, padding='post')
  
  return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar


In [ ]:
num_examples = len(temp)
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(file_path, num_examples)


In [ ]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)


In [ ]:
num_examples = len(temp)
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(file_path, num_examples)


##Google

In [55]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 1.0MB 38.8MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=60dd974400e6edb8302f925d1b28ba18518f67dade8f5603ddb5b38940746b28
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [ ]:
from googletrans import Translator
detector = Translator()



In [57]:
word = transliterate("teraa")

In [58]:

dec_lan = detector.translate(word)

print(dec_lan)


Translated(src=hi, dest=en, text=Tera, pronunciation=None, extra_data="{'translat...")


In [59]:
print(dec_lan.text)


Tera


##take 2

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.executing_eagerly()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

### Download dataset

Dataset source : http://www.manythings.org/anki/. 

As per the requirement, I am using Hin-Eng file from the list. 

file_path ="hin.txt"

# For testing if the file is loaded successfully, we can print few lines from the file using:
temp = open(file_path, encoding='UTF-8').read().strip().split('\n')
temp[0:5]
len(temp)

# We need to convert unicode file to ascii

def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  
  # We need to add space between every word and punctuation such as : "Wow! It is done." -> "Wow ! It is done ."
  # This is because out network recognises these characters as different words.
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
    
  # We also need every thing with space except alphabets, punctuations and assisting symbols
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.rstrip().strip()
  
  # Adding a start and end token to the sentence
  w = '<start> ' + w + ' <end>'
  return w

# Few preprocessing tasks such as:
# 1. Removing accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, HINDI]

def create_dataset(path, num_examples):
  lines =  open(path, encoding='UTF-8').read().strip().split('\n')
  
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]
  
  return word_pairs

# Creating word-index mapping (e.g. "wow"->7) for both the language.

class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))

    self.vocab = sorted(self.vocab)

    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1

    for word, index in self.word2idx.items():
      self.idx2word[index] = word

def max_length(tensor):
  return max(len(t) for t in tensor)

def load_dataset(path, num_examples):
  # creating cleaned input and output pairs
  pairs = create_dataset(path, num_examples)
  
  # index language using the class defined above
  inp_lang = LanguageIndex(hi for en, hi in pairs)
  targ_lang = LanguageIndex(en for en, hi in pairs)
  
  # Vectorize the input and target languages
  
  # Hindi sentences
  input_tensor = [[inp_lang.word2idx[s] for s in hi.split(' ')] for en, hi in pairs]
  
  # English sentences
  target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
  
  # We set input vector size to the size of longest sentence (sentence with maximum length) in out dataset.
  max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
  # We need to add padding other sentences before using them. We will pad the input and output tensor to max-length
  input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, maxlen=max_length_inp, padding='post')
  
  target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, maxlen=max_length_tar, padding='post')
  
  return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

num_examples = len(temp)
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(file_path, num_examples)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

### Writing Encoder and Decoder model

The model will be an Attention model

def gru(units):
  return tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = gru(self.enc_units)

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)        
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = gru(self.dec_units)
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.W1 = tf.keras.layers.Dense(self.dec_units)
    self.W2 = tf.keras.layers.Dense(self.dec_units)
    self.V = tf.keras.layers.Dense(1)
        
  def call(self, x, hidden, enc_output):
    #  enc_output shape == (batch_size, max_length, hidden_size)
        
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
    score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * enc_output
    context_vector = tf.reduce_sum(context_vector, axis=1)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size * 1, vocab)
    x = self.fc(output)

    return x, state, attention_weights

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.dec_units))    

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### Defining optimizer and loss function

optimizer = tf.optimizers.Adam()

def loss_function(real, pred):
  mask = 1-np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

# saving the checkpoints
checkpoint_dir = checkpoint_prefix = os.path.join('', "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### Training

EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()
    
  hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset):
    loss = 0
        
    with tf.GradientTape() as tape:
      enc_output, enc_hidden = encoder(inp, hidden)

      dec_hidden = enc_hidden

      dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       

      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    total_loss += batch_loss

    variables = encoder.variables + decoder.variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  

def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

    # storing the attention weigths to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.idx2word[predicted_id] + ' '

    if targ_lang.idx2word[predicted_id] == '<end>':
        return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  plt.show()

def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
  result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

  print('Input: {}'.format(sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

translate('मैं ठीक हूँ', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

#CHALLENGES

Our biggest challenge was to have a limited corpus for Hinglish words, which posed as a major setback for translating uncommon words to Hindi and English. Also, sice the corpus consisted of just words and not sentences, we could not translate Hinglish sentences to proper English or Hindi. Working with tensorflow was also a challenge because of the outdated libraries as well as reduced usage of tensorflow-core, which made it difficult to debug. Lastly, Hinglish being a casually-made language has no fixed spellings of any word. This increased the scope of error during translation.

English is based on Subject-Verb-Object (SVO) structure, but
Hindi is an Subject-Object-Verb (SOV) type of language.
Hindi is morphologically more rich than English. In general,
these divergences are the factors which make the translation
process difficult and error-prone. Furthermore, Hindi also have
some inherent challenges in translating to English (1) Lack of
articles in Hindi makes the translation imprecise. (2) Multiple
contextual meaning of English